In [1]:
from neuron import h, units, rxd, load_mechanisms
from neuron.units import mV, ms, um, mM
import matplotlib
from matplotlib import pyplot as plt
h.load_file("stdrun.hoc")
%matplotlib inline
from itertools import product
import plotly.express as px
import pandas as pd
import os

In [2]:
# This function assumes the ECS is a cube and each of the voxels are cubes
# It calculates how many voxels span across on of the sides of the cube
# The possible indicies for the voxels will the set containing all possible
# combinations of the numbers from 0 to (length in voxels - 1)
# This comes out as a iterable object of tuples, so we convert it and each
# of its elements to lists.

def create_index_list(species, ecs_name):
    length_in_voxels = len(species[ecs_name].states3d[0][0])
    coord_list = list(product(range(length_in_voxels), repeat = 3))
    coord_list = [list(coord) for coord in coord_list]
    return coord_list

def create_rec_conc_vects(time_vect_name, coord_list, species, ecs_name):
    vect_dict = {time_vect_name : h.Vector().record(h._ref_t)}
    num_voxels = len(coord_list)
    for i in range(num_voxels):
        vect_dict[str(species) + str(coord_list[i])] = h.Vector()
        vect_dict[str(species) + str(coord_list[i])].record(species[ecs_name].node_by_ijk(coord_list[i][0], coord_list[i][1], coord_list[i][2])._ref_value)
    return vect_dict

def voxel_conc_plots(time_vect_name, coord_list, vector_dictionary, species, image_directory):
    os.mkdir("Plots/" + image_directory)
    num_voxels = len(coord_list)
    for i in range(num_voxels):
        plt.plot(vector_dictionary[time_vect_name], vector_dictionary[str(species) + str(coord_list[i])])
        plt.title(str(species) + str(coord_list[i]))
        plt.savefig("Plots/" + image_directory + "/" + str(species) + str(coord_list[i]) + ".png")
        plt.close()

def run_and_visualize_sim(species, ecs_name, time_vect_name, simulation_time, image_directory):
    coord_list = create_index_list(species, ecs_name)
    vector_dictionary = create_rec_conc_vects(time_vect_name, coord_list, species, ecs_name)
    h.finitialize()
    h.continuerun(simulation_time * ms)
    voxel_conc_plots(time_vect_name, coord_list, vector_dictionary, species, image_directory)
    print(min(vector_dictionary[str(species) + "[1, 1, 1]"]), max(vector_dictionary[str(species) + "[1, 1, 1]"])) 

In [3]:
my_cell = h.Section(name = "my_cell")
my_cell.insert("one")

my_cell

In [4]:
# t_vec = h.Vector().record(h._ref_t)
JIS = h.Vector()
JIS.record(my_cell(0.5).one._ref_JIS)
I = h.Vector()
I.record(my_cell(0.5).one._ref_I)

Vector[1]

In [5]:
my_cell2 = h.Section(name = "my_cell2")
my_cell2.pt3dadd(10,15,15,10)
my_cell2.pt3dadd(20,15,15,10)

In [6]:
# Create extracellular space that is a 30x30x30 cube made up of 10x10x10 voxels 
ecs = rxd.Extracellular(0,0,0,30,30,30, dx = (10, 10, 10))

# Leave d = 0 so no diffusion meaning no insulin should flow from one voxel to another
# The concentration curves of all voxels should be the same
test = rxd.Species(ecs, name = "test", d = 0, initial = 48.04298494148047, atolscale=1e-12)

In [7]:
vc = 1e-13
fb = 2000
coords_test = create_index_list(test, ecs)
vecs = create_rec_conc_vects("t_vec", coords_test, test, ecs)
test_secretion = rxd.Rate(test, JIS/vc-fb*test, regions = ecs)

In [9]:
h.finitialize()
h.continuerun(2000 * ms)

FileNotFoundError: Could not find module 'c:\Users\rober\OneDrive - University of Florida\Documents\Research\Pancreatic_islet_model\Model-of-Pancreatic-Islets\Test\rxddll3ecdb187-038c-11ec-857b-c858c0244ed2.so' (or one of its dependencies). Try using the full path with constructor syntax.

NEURON: Python Callback failed
 near line 0
 ^
        finitialize()


RuntimeError: hoc error

In [ ]:
plt.plot(t_vec, I)

In [20]:
voxel_conc_plots("t_vec2", coords_test, vecs, test, "JIS_vector_rxd_test")